In [1]:
### THIS SCRIPT CLEANS THE COLLECTED SEP ###   

In [9]:
import pymongo 
import requests
import json
import pandas as pd
import re

from tqdm import tqdm
from bson.objectid import ObjectId
from pprint import pprint

import lib_sepinpho as sep

##### INIT GLOBAL VARIABLES#####

#init Mongo
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

#create database
db = client.visualizing_sep

In [6]:
##### Cleaning Sep_Entries ########
##### Fall Edition: Completed 10.1.2020 ########

#1. Check for entries with InPhO apis.

collection_to_clean = db.sep_entries

sep_no_inpho = pd.DataFrame(collection_to_clean.find( filter={'inpho_api': 'Error: No InPhO entry'},
                                         projection = ['title'],
                                         sort=[('title',1)]
                                        ))
#export to csv to made edits in this file
sep_no_inpho.to_csv('noinpho.csv')


In [49]:
# read the values to update from the CSV and batch update the entries

collection_to_update = db.sep_entries

#import cleaned data
inpho_updates = pd.read_csv('noinpho.csv')

for index,row in tqdm(inpho_updates.iterrows()):
    title = row['title']
    api_endpoint = row['api_endpoint']
    json_type = row['json_type']

    sep.update_sep_json(title,api_endpoint, json_type, collection_to_update)



1it [00:00,  1.80it/s]acknowledged: http://inpho.cogs.indiana.edu/thinker/2553.json
 True
[{&#39;_id&#39;: ObjectId(&#39;5f1bafb6896f82cdbda60f27&#39;), &#39;title&#39;: &#39;Aristotle&#39;, &#39;inpho_api&#39;: &#39;http://inpho.cogs.indiana.edu/thinker/2553.json&#39;, &#39;inpho_json&#39;: {&#39;wiki&#39;: &#39;Aristotle&#39;, &#39;influenced&#39;: [2462, 2472, 2477, 2493, 2543, 2549, 2576, 2577, 2671, 2685, 2731, 2768, 2905, 2943, 2953, 2985, 3021, 3049, 3114, 3132, 3177, 3179, 3223, 3295, 3367, 3383, 3404, 3439, 3476, 3511, 3550, 3580, 3585, 3590, 3597, 3657, 3738, 3760, 3776, 3777, 3805, 3842, 3911, 3936, 3950, 3963, 3974, 3975, 3991, 4006, 4021, 4025, 4077, 4189], &#39;death&#39;: [{&#39;month&#39;: 3, &#39;day&#39;: 7, &#39;year&#39;: -322}], &#39;teachers&#39;: [3724], &#39;url&#39;: &#39;/thinker/2553&#39;, &#39;influenced_by&#39;: [3191, 3682, 3724, 3919], &#39;professions&#39;: [&#39;Student&#39;, &#39;Teacher&#39;], &#39;death_strings&#39;: [&#39;March 7, 322 B.C.E.&#39;], 

In [50]:
##### Cleaning Sep_Entries ########
##### Summer: Completed 7.16.2020 ########
##### Summer: Completed 10.1.2020 ########

#2. Check for duplicate InPhO JSON data.

collection_to_clean = db.sep_entries
result = list(collection_to_clean.aggregate([
                                {
                                    '$group' : { '_id' : "$inpho_json.ID",
                                    'count': { '$sum': 1 }
                                    }
                                },
                                {
                                    '$match': { 'count': {'$ne': 1 }}
                                }]))

pprint(result)

[{&#39;_id&#39;: None, &#39;count&#39;: 11}]


In [51]:

#Find any article that doesn't have domain info

collection_to_update = db.sep_entries

no_domains = list(collection_to_update.find(
    {'primary_domain': {'$exists': False}},
    projection={'page_url':1, 'title':1, '_id':0}
))

pprint(no_domains)

[{&#39;page_url&#39;: &#39;/entries/albalag/&#39;, &#39;title&#39;: &#39;Isaac Albalag&#39;},
 {&#39;page_url&#39;: &#39;/entries/alternative-possibilities/&#39;,
  &#39;title&#39;: &#39;Moral Responsibility and the Principle of Alternative &#39;
           &#39;Possibilities&#39;},
 {&#39;page_url&#39;: &#39;/entries/boethius-dacia/&#39;, &#39;title&#39;: &#39;Boethius of Dacia&#39;},
 {&#39;page_url&#39;: &#39;/entries/causation-physics/&#39;, &#39;title&#39;: &#39;Causation in Physics&#39;},
 {&#39;page_url&#39;: &#39;/entries/divine-revelation/&#39;, &#39;title&#39;: &#39;Divine Revelation&#39;},
 {&#39;page_url&#39;: &#39;/entries/evolution-development/&#39;,
  &#39;title&#39;: &#39;Evolution and Development&#39;},
 {&#39;page_url&#39;: &#39;/entries/luther-influence/&#39;,
  &#39;title&#39;: &#39;Luther’s Influence on Philosophy&#39;},
 {&#39;page_url&#39;: &#39;/entries/luther/&#39;, &#39;title&#39;: &#39;Martin Luther&#39;},
 {&#39;page_url&#39;: &#39;/entries/microbiology/&#39

In [6]:
#take the list of articles in no_domains, and make a list of the domain data to add for each of them. 

domains_to_update = [
    ('Thinker', 'Thinker'),
    ('Ethics and Morality', 'Ethics and Morality', 'Metaphysics'),
    ('Thinker', 'Thinker'),
    ('Physics', 'Physics', 'Scientific Methods', 'Metaphysics'),
    ('Religion', 'Religion, Epistemology'),
    ('Evolution', 'Evolution, Biology'),
    ('Thinker', 'Thinker, Religion'),
    ('Thinker', 'Thinker, Religion'),
    ('Biology', 'Biology'),
    ('Thinker', 'Thinker'),
    ('Mind', 'Mind, Biology')
]

#check to make sure domains_to_update and no_domains are the same size
print(len(no_domains))
print(len(domains_to_update))

11
11


In [52]:
#update articles in no_domains with new domain data  
#Fall Update: Completed 10.1.2020#
collection_to_update = db.sep_entries

for i, articles in enumerate(no_domains):
    article_to_update = articles['page_url']
    primary_domain = domains_to_update[i][0]
    domain_tags = domains_to_update[i][1] 
    new_values = {
         'primary_domain': primary_domain,
         'domain_tags': domain_tags
         }
    result = collection_to_update.update_one(
        {'page_url': article_to_update},
        {'$set': new_values}
    )
    # boolean confirmation that the API call went through
    print (f'acknowledged: {article_to_update}\n', result.acknowledged)
    


acknowledged: /entries/albalag/
 True
acknowledged: /entries/alternative-possibilities/
 True
acknowledged: /entries/boethius-dacia/
 True
acknowledged: /entries/causation-physics/
 True
acknowledged: /entries/divine-revelation/
 True
acknowledged: /entries/evolution-development/
 True
acknowledged: /entries/luther-influence/
 True
acknowledged: /entries/luther/
 True
acknowledged: /entries/microbiology/
 True
acknowledged: /entries/protagoras/
 True
acknowledged: /entries/psychology-normative-cognition/
 True


In [53]:
#now test again for no domain info

collection_to_update = db.sep_entries

no_domains = list(collection_to_update.find(
    {'primary_domain': {'$exists': False}},
    projection={'page_url':1, 'title':1, '_id':0}
))

pprint(no_domains)

[]


In [54]:
# update domain tags and primary domain for all articles 
# completed 10.1.2020

collection_to_update = db.sep_entries

collection_to_update.create_index([('preamble_text','text'), ('main_text','text')], name='preamble_textindex')
collection_to_update.index_information()



{&#39;_id_&#39;: {&#39;v&#39;: 2, &#39;key&#39;: [(&#39;_id&#39;, 1)], &#39;ns&#39;: &#39;visualizing_sep.sep_entries&#39;},
 &#39;title_1&#39;: {&#39;v&#39;: 2,
  &#39;key&#39;: [(&#39;title&#39;, 1)],
  &#39;ns&#39;: &#39;visualizing_sep.sep_entries&#39;},
 &#39;preamble_textindex&#39;: {&#39;v&#39;: 2,
  &#39;key&#39;: [(&#39;_fts&#39;, &#39;text&#39;), (&#39;_ftsx&#39;, 1)],
  &#39;ns&#39;: &#39;visualizing_sep.sep_entries&#39;,
  &#39;weights&#39;: SON([(&#39;main_text&#39;, 1), (&#39;preamble_text&#39;, 1)]),
  &#39;default_language&#39;: &#39;english&#39;,
  &#39;language_override&#39;: &#39;language&#39;,
  &#39;textIndexVersion&#39;: 3}}

In [3]:
pprint(collection_to_update.distinct('primary_domain'))

NameError: name &#39;collection_to_update&#39; is not defined

In [14]:
collection_to_update = db.sep_entries
all_domains = pd.DataFrame(collection_to_update.find({},
                                              {'title':1, 'author': 1, 'primary_domain':1, 'domain_tags':1}))
all_domains.to_csv('domains.csv')

In [18]:
collection_to_update = db.sep_entries

#import updated data
updated_domains = pd.read_csv('domains.csv')

for index,row in tqdm(updated_domains.iterrows()):
    title = row['title']
    domain_tags = row['domain_tags']
    primary_domain = row['primary_domain']

    sep.update_domain_info(title,domain_tags, primary_domain, collection_to_update)

imary_domain&#39;: &#39;Logic&#39;}]
acknowledged: Typelogical Grammar
 True
[{&#39;_id&#39;: ObjectId(&#39;5f1bb9b4896f82cdbda6150f&#39;), &#39;title&#39;: &#39;Typelogical Grammar&#39;, &#39;domain_tags&#39;: &#39;Logic,Philosophy of Language&#39;, &#39;primary_domain&#39;: &#39;Logic&#39;}]
acknowledged: Types and Tokens
 True
[{&#39;_id&#39;: ObjectId(&#39;5f1bb9b6896f82cdbda61510&#39;), &#39;title&#39;: &#39;Types and Tokens&#39;, &#39;domain_tags&#39;: &#39;Metaphysics&#39;, &#39;primary_domain&#39;: &#39;Metaphysics&#39;}]
acknowledged: Umar Khayyam
 True
[{&#39;_id&#39;: ObjectId(&#39;5f1bb9b7896f82cdbda61511&#39;), &#39;title&#39;: &#39;Umar Khayyam&#39;, &#39;domain_tags&#39;: &#39;Thinker,Arabic and Islamic Philosophy&#39;, &#39;primary_domain&#39;: &#39;Thinker&#39;}]
acknowledged: The Medieval Problem of Universals
 True
[{&#39;_id&#39;: ObjectId(&#39;5f1bb9b9896f82cdbda61512&#39;), &#39;title&#39;: &#39;The Medieval Problem of Universals&#39;, &#39;domain_tags&#39;: &#39;

In [4]:
collection_to_update = db.sep_entries
pprint(collection_to_update.distinct('primary_domain'))

[&#39;Aesthetics and Philosophy of Art&#39;,
 &#39;African and African-American Philosophy&#39;,
 &#39;Arabic and Islamic Philosophy&#39;,
 &#39;Chinese Philosophy&#39;,
 &#39;Continental Philosophy&#39;,
 &#39;Epistemology&#39;,
 &#39;Ethics&#39;,
 &#39;Feminist Philosophy&#39;,
 &#39;Indian Philosophy&#39;,
 &#39;Japanese Philosophy&#39;,
 &#39;Latin American Philosophy&#39;,
 &#39;Logic&#39;,
 &#39;Metaphysics&#39;,
 &#39;Philosophy of Biology&#39;,
 &#39;Philosophy of Computer Science&#39;,
 &#39;Philosophy of Economics&#39;,
 &#39;Philosophy of Language&#39;,
 &#39;Philosophy of Law&#39;,
 &#39;Philosophy of Mathematics&#39;,
 &#39;Philosophy of Mind&#39;,
 &#39;Philosophy of Physics&#39;,
 &#39;Philosophy of Religion&#39;,
 &#39;Philosophy of Science&#39;,
 &#39;Political and Social Theory&#39;,
 &#39;Social and Political Philosophy&#39;,
 &#39;Thinker&#39;]


In [ ]:
#Prior Updates
# African American Philosophy: Completed 10.2.2020
# text_filter = 'African African-American'
# sep_filter = 'African and African-American Philosophy'
#########################################################




In [5]:
#update domains

collection_to_update = db.sep_entries

text_filter = 'African African-American'
sep_filter = 'African and African-American Philosophy'

text_results = pd.DataFrame(collection_to_update.find(
                                         {'$text':{'$search': text_filter }},
                                         {'score': { '$meta': 'textScore'}, 
                                            'title':1, 
                                            'page_url':1,
                                            'primary_domain':1, 
                                            'domain_tags':1}
                                         ))
text_results_above1 = text_results[text_results.score > 1]

text_results_above1.to_csv('aa1.csv')

sep_results = pd.DataFrame(collection_to_update.find({ 'domain_tags': { '$regex': sep_filter } },
                                              {'title':1, 'primary_domain':1, 'domain_tags':1}))

sep_results.to_csv('aa_sep.csv')

In [12]:

collection_to_update = db.sep_entries
domains_to_update = pd.read_csv('aa1.csv')
domain_to_update.head()

for index,row in tqdm(domain_to_update.iterrows()):
    title = row['title']
    domain_tags = row['domain_tags']
    primary_domain = row['primary_domain']
    page_url = row['page_url']

    sep.update_domain_info(page_url,domain_tags, primary_domain, collection_to_update)



23it [00:00, 221.75it/s]acknowledged: /entries/justice-intergenerational/
 True
[{&#39;_id&#39;: ObjectId(&#39;5f1bb47a896f82cdbda611d5&#39;), &#39;title&#39;: &#39;Intergenerational Justice&#39;, &#39;domain_tags&#39;: &#39;Social and Political Philosophy, African and African-American Philosophy&#39;, &#39;primary_domain&#39;: &#39;Social and Political Philosophy&#39;}]
acknowledged: /entries/recognition/
 True
[{&#39;_id&#39;: ObjectId(&#39;5f1bb80f896f82cdbda6140c&#39;), &#39;title&#39;: &#39;Recognition&#39;, &#39;domain_tags&#39;: &#39;Philosophy of Mind, Ethics, Social and Political Philosophy&#39;, &#39;primary_domain&#39;: &#39;Social and Political Philosophy&#39;}]
acknowledged: /entries/dirty-hands/
 True
[{&#39;_id&#39;: ObjectId(&#39;5f1bb1d8896f82cdbda61048&#39;), &#39;title&#39;: &#39;The Problem of Dirty Hands&#39;, &#39;domain_tags&#39;: &#39;Ethics,Social and Political Philosophy&#39;, &#39;primary_domain&#39;: &#39;Social and Political Philosophy&#39;}]
acknowledged: 

In [13]:

update_results = list(collection_to_update.find({ 'domain_tags': { '$regex': 'African'} },
                                              {'title':1, 'primary_domain':1, 'domain_tags':1}))

pprint(update_results)

[{&#39;_id&#39;: ObjectId(&#39;5f1baf35896f82cdbda60ed8&#39;),
  &#39;domain_tags&#39;: &#39;Social and Political Philosophy, African and &#39;
                 &#39;African-American Philosophy&#39;,
  &#39;primary_domain&#39;: &#39;Social and Political Philosophy&#39;,
  &#39;title&#39;: &#39;Affirmative Action&#39;},
 {&#39;_id&#39;: ObjectId(&#39;5f1baf36896f82cdbda60ed9&#39;),
  &#39;domain_tags&#39;: &#39;African and African-American Philosophy,Ethics&#39;,
  &#39;primary_domain&#39;: &#39;African and African-American Philosophy&#39;,
  &#39;title&#39;: &#39;African Ethics&#39;},
 {&#39;_id&#39;: ObjectId(&#39;5f1baf38896f82cdbda60eda&#39;),
  &#39;domain_tags&#39;: &#39;African and African-American Philosophy,Social and Political &#39;
                 &#39;Philosophy&#39;,
  &#39;primary_domain&#39;: &#39;African and African-American Philosophy&#39;,
  &#39;title&#39;: &#39;African Sage Philosophy&#39;},
 {&#39;_id&#39;: ObjectId(&#39;5f1baf3a896f82cdbda60edb&#39;),
  &#39;domai